# Imports

In [ ]:
import os
import glob
import shutil

# Setup

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

In [ ]:
# !apt-get install lz4

In [ ]:
# %%capture
# !pip install --upgrade duckdb papermill s5cmd dcm2niix pyplastimatch pyradiomics \
# highdicom pydicom seg-metrics idc-index itk SimpleITK pyaml

In [ ]:
# %%capture
# from pyplastimatch.utils.install import install_precompiled_binaries
# install_precompiled_binaries()

In [ ]:
# dcmqi_release_url = "https://github.com/QIICR/dcmqi/releases/download/v1.3.4/dcmqi-1.3.4-linux.tar.gz"
# dcmqi_download_path = "dcmqi-1.3.4-linux.tar.gz"
# dcmqi_path = "dcmqi-1.3.4-linux"
# !wget -O $dcmqi_download_path $dcmqi_release_url\
# && tar -xvf $dcmqi_download_path\
# && mv $dcmqi_path/bin/* /bin\
# && rm -r $dcmqi_download_path $dcmqi_path

# Global Variables


In [ ]:
#absolute path output
OUTPUT_PATH = ""#"radiomics_task"#""
#input MHUB zip files
mhubCompressedOutputFiles = []
#input rads ai compressed zip files
radsAiCompressedOutputFiles = []
#input eval compressed zip files
evalCompressedOutputFiles = []
#second list eval
evalAddCompressedOutputFiles = []
#input IDC expert annotations zip file
idcExpertCompressedOutputFile = ""
#input IDC expert SR annotations zip file
radsIdcExpertCompressedOutputFile = ""
#second list IDC annotations files
idcExpertAddCompressedOutputFile = ""
radsIdcExpertAddCompressedOutputFile = ""
#list of mhub_models
mhub_model_name_lst = []

In [ ]:
# #absolute path output
# OUTPUT_PATH = "combine_task"#"radiomics_task"#""
# #input MHUB zip files
# mhubCompressedOutputFiles = 	[ "gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/submissions/366f729d-713d-481b-bc94-e5a67104264d/mhubai_workflow/48ebf735-e98c-4c21-9b70-4e8e768c4b0a/call-ai_combine_seg/shard-0/ai_combination_archive.tar.lz4",
#                               "gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/submissions/366f729d-713d-481b-bc94-e5a67104264d/mhubai_workflow/48ebf735-e98c-4c21-9b70-4e8e768c4b0a/call-ai_combine_seg/shard-1/ai_combination_archive.tar.lz4" ]
# #input rads ai compressed zip files
# radsAiCompressedOutputFiles = 	[ "gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/submissions/366f729d-713d-481b-bc94-e5a67104264d/mhubai_workflow/48ebf735-e98c-4c21-9b70-4e8e768c4b0a/call-ai_rads/shard-0/radiomics_archive.tar.lz4",
#                                 "gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/submissions/366f729d-713d-481b-bc94-e5a67104264d/mhubai_workflow/48ebf735-e98c-4c21-9b70-4e8e768c4b0a/call-ai_rads/shard-1/radiomics_archive.tar.lz4" ]
# #input eval compressed zip files
# evalCompressedOutputFiles = [ "gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/submissions/366f729d-713d-481b-bc94-e5a67104264d/mhubai_workflow/48ebf735-e98c-4c21-9b70-4e8e768c4b0a/call-evaluate_ai/shard-0/eval_archive.tar.lz4",
#                              "gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/submissions/366f729d-713d-481b-bc94-e5a67104264d/mhubai_workflow/48ebf735-e98c-4c21-9b70-4e8e768c4b0a/call-evaluate_ai/shard-1/eval_archive.tar.lz4" ]
# #input IDC expert annotations zip file
# idcExpertCompressedOutputFile = "gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/submissions/366f729d-713d-481b-bc94-e5a67104264d/mhubai_workflow/48ebf735-e98c-4c21-9b70-4e8e768c4b0a/call-idc_combine_seg/idc_combination_archive.tar.lz4"
# #input IDC expert SR annotations zip file
# radsIdcExpertCompressedOutputFile = "gs://fc-5af492dc-6993-4c91-bbf6-3e2747868642/submissions/366f729d-713d-481b-bc94-e5a67104264d/mhubai_workflow/48ebf735-e98c-4c21-9b70-4e8e768c4b0a/call-idc_rads/radiomics_archive.tar.lz4"
# #list of mhub_models
# mhub_model_name_lst = ["bamf_nnunet_mr_prostate", "bamf_nnunet_mr_prostate_2"]

# Paths definition

In [ ]:
TASK_OUT_UNZIP = os.path.join(OUTPUT_PATH, "tasks_unzip")
#mhub_outputs
mhub_seg_out_unzip = os.path.join(TASK_OUT_UNZIP, "mhub_seg")
mhub_sr_out_unzip = os.path.join(TASK_OUT_UNZIP, "mhub_sr")
#eval outputs
eval_seg_out_unzip = os.path.join(TASK_OUT_UNZIP, "eval")
##second set of annotations
eval_seg_add_out_unzip = os.path.join(TASK_OUT_UNZIP, "eval_add")
#idc experts outputs
idc_expert_seg_out_unzip = os.path.join(TASK_OUT_UNZIP, "idc_expert_seg")
idc_expert_sr_out_unzip =os.path.join(TASK_OUT_UNZIP, "idc_expert_sr")
#second set of idc experts outputs
idc_expert_seg_add_out_unzip = os.path.join(TASK_OUT_UNZIP, "idc_expert_seg_add")
idc_expert_sr_add_out_unzip =os.path.join(TASK_OUT_UNZIP, "idc_expert_sr_add")

#final folder path
base_out_folder = os.path.join(OUTPUT_PATH, "aggregated_results")
##AI output folders
ai_seg_out_folder = os.path.join(base_out_folder, "ai_results", "dicom_seg")
ai_sr_out_folder = os.path.join(base_out_folder, "ai_results", "dicom_sr")
##IDC output folders
idc_seg_out_folder = os.path.join(base_out_folder, "idc_expert_results", "dicom_seg")
idc_sr_out_folder = os.path.join(base_out_folder, "idc_expert_results", "dicom_sr")
##second set of segmentations
idc_seg_add_out_folder = os.path.join(base_out_folder, "idc_expert_results_add", "dicom_seg")
idc_sr_add_out_folder = os.path.join(base_out_folder, "idc_expert_results_add", "dicom_sr")
##eval folders
eval_out_folder = os.path.join(base_out_folder, "quant_eval_results")
eval_add_out_folder = os.path.join(base_out_folder, "quant_eval_results_add")
##papermill notebooks log outputs
papermill_logs_out_folder = os.path.join(base_out_folder, "papermill_logs")

In [ ]:
def setup_folders():
  """
  creates environment data and results folder, first deletes current ones and then creates new ones
  """
  for folder_setup in [TASK_OUT_UNZIP, mhub_seg_out_unzip,
                       mhub_sr_out_unzip, eval_seg_out_unzip,
                       idc_expert_seg_out_unzip, idc_expert_sr_out_unzip,
                       base_out_folder, ai_seg_out_folder,
                       ai_sr_out_folder, idc_seg_out_folder,
                       idc_sr_out_folder,eval_out_folder,
                       eval_add_out_folder,
                       papermill_logs_out_folder,
                       eval_seg_add_out_unzip, 
                       idc_expert_seg_add_out_unzip,
                       idc_expert_sr_add_out_unzip]:
    !rm -rf $folder_setup
    !mkdir -p $folder_setup

In [ ]:
setup_folders()

# Create model-specific output folders

In [ ]:
for model_name in mhub_model_name_lst:
  #create models unzip paths
  for path_model_create in [mhub_seg_out_unzip, mhub_sr_out_unzip,
                            eval_seg_out_unzip, eval_add_out_folder]:
    temp_path = os.path.join(path_model_create, model_name)
    !mkdir -p $temp_path
  ##create models out paths
  for path_model_create in [ai_seg_out_folder, ai_sr_out_folder,
                            eval_out_folder, papermill_logs_out_folder]:
    temp_path = os.path.join(path_model_create, model_name)
    !mkdir -p $temp_path

# Unzip all outputs and aggregate into final folder

mhub ai segs -- Unzip and move to output directory

In [ ]:
for idx_model, model_name in enumerate(mhub_model_name_lst):
  #download archive
  archive_temp = mhubCompressedOutputFiles[idx_model]
  archive_filemame = archive_temp.split("/")[-1]
  zip_ddl_path = os.path.join(mhub_seg_out_unzip, model_name)
  # !gsutil -m cp -r $archive_temp $zip_ddl_path
  !cp $archive_temp $zip_ddl_path
  zip_path = os.path.join(mhub_seg_out_unzip, model_name, archive_filemame)
  !lz4 -dc < $zip_path | tar xvf - -C $zip_ddl_path
  #move archive contents to output folder
  dcm_files_lst = glob.glob(os.path.join(zip_ddl_path, "**", "*.dcm"), recursive=True)
  ##get image_serieUID
  ##define output path for each DCM file, add to the .dcm  a prefix image_serieUID
  for in_dicom_file_path in dcm_files_lst:
    image_segmented_serieUID = in_dicom_file_path.split("/")[-2].strip()
    ##make sure output_path_folder is defined
    out_folder_path = os.path.join(ai_seg_out_folder, model_name,
                 image_segmented_serieUID)
    !mkdir -p $out_folder_path
    out_dcm_file_path = os.path.join(out_folder_path,
                                     image_segmented_serieUID+"_"+in_dicom_file_path.split("/")[-1].strip())
    #copy to output folder
    !cp $in_dicom_file_path $out_dcm_file_path

Unzip mhub ai srs

In [ ]:
for idx_model, model_name in enumerate(mhub_model_name_lst):
  archive_temp = radsAiCompressedOutputFiles[idx_model]
  archive_filemame = archive_temp.split("/")[-1]
  zip_ddl_path = os.path.join(mhub_sr_out_unzip, model_name)
  # !gsutil -m cp -r $archive_temp $zip_ddl_path
  !cp $archive_temp $zip_ddl_path
  zip_path = os.path.join(mhub_sr_out_unzip, model_name, archive_filemame)
  !lz4 -dc < $zip_path | tar xvf - -C $zip_ddl_path
  #move archive contents to output folder
  dcm_files_lst = glob.glob(os.path.join(zip_ddl_path, "**", "*.dcm"), recursive=True)
  ##get image_serieUID
  ##define output path for each DCM file, add to the .dcm  a prefix image_serieUID
  for in_dicom_file_path in dcm_files_lst:
    image_segmented_serieUID = in_dicom_file_path.split("/")[-2].strip()
    ##make sure output_path_folder is defined
    out_folder_path = os.path.join(ai_sr_out_folder, model_name,
                 image_segmented_serieUID)
    !mkdir -p $out_folder_path
    out_dcm_file_path = os.path.join(out_folder_path,
                                     image_segmented_serieUID+"_"+in_dicom_file_path.split("/")[-1].strip())
    #copy to output folder
    !cp $in_dicom_file_path $out_dcm_file_path

Unzip quant eval results

In [ ]:
for idx_model, model_name in enumerate(mhub_model_name_lst):
  archive_temp = evalCompressedOutputFiles[idx_model]
  archive_filemame = archive_temp.split("/")[-1]
  zip_ddl_path = os.path.join(eval_seg_out_unzip, model_name)
  # !gsutil -m cp -r $archive_temp $zip_ddl_path
  !cp $archive_temp $zip_ddl_path
  zip_path = os.path.join(eval_seg_out_unzip, model_name, archive_filemame)
  !lz4 -dc < $zip_path | tar xvf - -C $zip_ddl_path
  #move archive contents to output folder
  eval_file_path = glob.glob(os.path.join(zip_ddl_path, "**", "output_eval.csv"), recursive=True)[0]
  ##get image_serieUID
  out_folder_path = os.path.join(eval_out_folder, model_name)
  !mkdir -p $out_folder_path
  out_eval_file_path = os.path.join(out_folder_path, eval_file_path.split("/")[-1].strip())
  #copy to output folder
  !cp $eval_file_path $out_eval_file_path

Unzip of eval results of second set of annotations

In [ ]:
for idx_model, model_name in enumerate(mhub_model_name_lst):
  archive_temp = evalAddCompressedOutputFiles[idx_model]
  archive_filemame = archive_temp.split("/")[-1]
  zip_ddl_path = os.path.join(eval_seg_add_out_unzip, model_name)
  # !gsutil -m cp -r $archive_temp $zip_ddl_path
  !cp $archive_temp $zip_ddl_path
  zip_path = os.path.join(eval_seg_add_out_unzip, model_name, archive_filemame)
  !lz4 -dc < $zip_path | tar xvf - -C $zip_ddl_path
  eval_file_path_lst = glob.glob(os.path.join(zip_ddl_path, "**", "output_eval.csv"), recursive=True)
  #move archive contents to output folder
  if len(eval_file_path_lst) > 0:
    eval_file_path = eval_file_path_lst[0]
    ##get image_serieUID
    out_folder_path = os.path.join(eval_add_out_folder, model_name)
    !mkdir -p $out_folder_path
    out_eval_file_path = os.path.join(out_folder_path, eval_file_path.split("/")[-1].strip())
    #copy to output folder
    !cp $eval_file_path $out_eval_file_path

Unzip idc expert annotations segs

In [ ]:
archive_temp = idcExpertCompressedOutputFile
archive_filemame = archive_temp.split("/")[-1]
zip_ddl_path = os.path.join(idc_expert_seg_out_unzip)
# !gsutil -m cp -r $archive_temp $zip_ddl_path
!cp $archive_temp $zip_ddl_path
zip_path = os.path.join(idc_expert_seg_out_unzip, archive_filemame)
!lz4 -dc < $zip_path | tar xvf - -C $zip_ddl_path
#move archive contents to output folder
dcm_files_lst = glob.glob(os.path.join(zip_ddl_path, "**", "*.dcm"), recursive=True)
##get image_serieUID
##define output path for each DCM file, add to the .dcm  a prefix image_serieUID
for in_dicom_file_path in dcm_files_lst:
  image_segmented_serieUID = in_dicom_file_path.split("/")[-2].strip()
  ##make sure output_path_folder is defined
  out_folder_path = os.path.join(idc_seg_out_folder,
                image_segmented_serieUID)
  !mkdir -p $out_folder_path
  out_dcm_file_path = os.path.join(out_folder_path,
                                    image_segmented_serieUID+"_"+in_dicom_file_path.split("/")[-1].strip())
  #copy to output folder
  !cp $in_dicom_file_path $out_dcm_file_path

Unzip second set of IDC expert annotations

In [ ]:
archive_temp = idcExpertAddCompressedOutputFile
archive_filemame = archive_temp.split("/")[-1]
zip_ddl_path = os.path.join(idc_expert_seg_add_out_unzip)
# !gsutil -m cp -r $archive_temp $zip_ddl_path
!cp $archive_temp $zip_ddl_path
zip_path = os.path.join(idc_expert_seg_add_out_unzip, archive_filemame)
!lz4 -dc < $zip_path | tar xvf - -C $zip_ddl_path
#move archive contents to output folder
dcm_files_lst = glob.glob(os.path.join(zip_ddl_path, "**", "*.dcm"), recursive=True)
##get image_serieUID
##define output path for each DCM file, add to the .dcm  a prefix image_serieUID
if len(dcm_files_lst) > 0:#second set of annotations might be empty
    for in_dicom_file_path in dcm_files_lst:
      image_segmented_serieUID = in_dicom_file_path.split("/")[-2].strip()
      ##make sure output_path_folder is defined
      out_folder_path = os.path.join(idc_seg_add_out_folder,
                    image_segmented_serieUID)
      !mkdir -p $out_folder_path
      out_dcm_file_path = os.path.join(out_folder_path,
                                        image_segmented_serieUID+"_"+in_dicom_file_path.split("/")[-1].strip())
      #copy to output folder
      !cp $in_dicom_file_path $out_dcm_file_path
else:
    pass

Unzip idc expert annotations srs

In [ ]:
archive_temp = radsIdcExpertCompressedOutputFile
archive_filemame = archive_temp.split("/")[-1]
zip_ddl_path = os.path.join(idc_expert_sr_out_unzip)
# !gsutil -m cp -r $archive_temp $zip_ddl_path
!cp $archive_temp $zip_ddl_path
zip_path = os.path.join(idc_expert_sr_out_unzip,archive_filemame)
!lz4 -dc < $zip_path | tar xvf - -C $zip_ddl_path
#move archive contents to output folder
dcm_files_lst = glob.glob(os.path.join(zip_ddl_path, "**", "*.dcm"), recursive=True)
##get image_serieUID
##define output path for each DCM file, add to the .dcm  a prefix image_serieUID
for in_dicom_file_path in dcm_files_lst:
  image_segmented_serieUID = in_dicom_file_path.split("/")[-2].strip()
  ##make sure output_path_folder is defined
  out_folder_path = os.path.join(idc_sr_out_folder,
                image_segmented_serieUID)
  !mkdir -p $out_folder_path
  out_dcm_file_path = os.path.join(out_folder_path,
                                    image_segmented_serieUID+"_"+in_dicom_file_path.split("/")[-1].strip())
  #copy to output folder
  !cp $in_dicom_file_path $out_dcm_file_path

Unzip second set of idc srs

In [ ]:
archive_temp = radsIdcExpertAddCompressedOutputFile
archive_filemame = archive_temp.split("/")[-1]
zip_ddl_path = os.path.join(idc_expert_sr_add_out_unzip)
# !gsutil -m cp -r $archive_temp $zip_ddl_path
!cp $archive_temp $zip_ddl_path
zip_path = os.path.join(idc_expert_sr_add_out_unzip,archive_filemame)
!lz4 -dc < $zip_path | tar xvf - -C $zip_ddl_path
#move archive contents to output folder
dcm_files_lst = glob.glob(os.path.join(zip_ddl_path, "**", "*.dcm"), recursive=True)
##get image_serieUID
##define output path for each DCM file, add to the .dcm  a prefix image_serieUID
if len(dcm_files_lst) > 0:#second set of annotations might be empty
    for in_dicom_file_path in dcm_files_lst:
      image_segmented_serieUID = in_dicom_file_path.split("/")[-2].strip()
      ##make sure output_path_folder is defined
      out_folder_path = os.path.join(idc_sr_add_out_folder,
                    image_segmented_serieUID)
      !mkdir -p $out_folder_path
      out_dcm_file_path = os.path.join(out_folder_path,
                                        image_segmented_serieUID+"_"+in_dicom_file_path.split("/")[-1].strip())
      #copy to output folder
      !cp $in_dicom_file_path $out_dcm_file_path
else:
    pass

Move papermill logs to archive

In [ ]:
for idx_model, model_name in enumerate(mhub_model_name_lst):
  #get papermill logs for ai_seg combine,ai_sr,eval_quant tasks
  papermill_combine_ai_seg_path = glob.glob(os.path.join(mhub_seg_out_unzip, model_name, "**", "*.ipynb"), recursive=True)[0]
  papermil_ai_sr_path = glob.glob(os.path.join(mhub_sr_out_unzip, model_name, "**", "*.ipynb"), recursive=True)[0]
  papermil_eval_path = glob.glob(os.path.join(eval_seg_out_unzip, model_name, "**", "*.ipynb"), recursive=True)[0]
  ##copy MHUB outputs and eval to archive folder
  out_papermil_ai_folder = os.path.join(papermill_logs_out_folder, model_name)
  !mkdir -p $out_papermil_ai_folder
  #copy to output folder
  ##define out paths
  out_papermil_ai_sr_file_path = os.path.join(out_papermil_ai_folder, papermil_ai_sr_path.split("/")[-1])
  out_papermil_ai_seg_file_path = os.path.join(out_papermil_ai_folder, papermill_combine_ai_seg_path.split("/")[-1])
  out_papermil_eval_file_path = os.path.join(out_papermil_ai_folder, papermil_eval_path.split("/")[-1])
  !cp $papermil_ai_sr_path $out_papermil_ai_sr_file_path
  !cp $papermill_combine_ai_seg_path $out_papermil_ai_seg_file_path
  !cp $papermil_eval_path $out_papermil_eval_file_path
#get papermill logs for idc seg combine, idc_sr tasks
papermil_idc_seg_path = glob.glob(os.path.join(idc_expert_seg_out_unzip, "**", "*.ipynb"), recursive=True)[0]
papermil_idc_sr_path = glob.glob(os.path.join(idc_expert_sr_out_unzip, "**", "*.ipynb"), recursive=True)[0]
out_papermill_idc_expert_path_folder =  os.path.join(papermill_logs_out_folder, "idc_expert_seg")
!mkdir -p $out_papermill_idc_expert_path_folder
out_papermil_idc_seg_path = os.path.join(out_papermill_idc_expert_path_folder,
                                       papermil_idc_seg_path.split("/")[-1])
out_papermil_idc_sr_path = os.path.join(out_papermill_idc_expert_path_folder,
                                      papermil_idc_sr_path.split("/")[-1])
#copy to output folder
!cp $papermil_idc_seg_path $out_papermil_idc_seg_path
!cp $papermil_idc_sr_path $out_papermil_idc_sr_path

# Create LZ4 archive

In [ ]:
output_lz4_archive = os.path.join(OUTPUT_PATH, "agg_archive.tar.lz4")

In [ ]:
# # Compress output data and move it to Cromwell root directory
!tar -C $OUTPUT_PATH -cvf - aggregated_results | lz4 > $output_lz4_archive